In [1]:
import matplotlib.pyplot as plt
import math
import numpy as np 
import pandas as pd 
import io 
import requests
import datetime as dt

In [ ]:
def dx(cap_VP_Air, MV_CanAir, MV_PadAir, MV_FogAir, MV_AirThScr, MV_AirTop, MV_AirOut, MV_AirOut_Pad, MV_AirTech, cap_VP_Top, MV_AirTop, MV_TopCov_in, MV_TopOut):
    global VP_Air
    cap_VP_Air = def_cap_VP_Air(M_water, h_Air, R, T_Air)
    print("cap_VPAir = ", cap_VP_Air)
    print()
    
    cap_VP_Top = def_cap_VP_Top(M_water, h_Gh, R, T_Top)
    print("cap_VP_Top = ", cap_VP_Top)
    print()
    
    rf_R_Can =  def_rf_R_can(R_Can, c_evap1, c_evap2)
    S_r_S = def_S_r_S(s_r_s, R_Can, R_Can_SP)
    c_evap3 = def_c_evap3(R_Can, R_Can_SP, night_c_evap3, day_c_evap3)
    c_evap4 = def_c_evap4(R_Can, R_Can_SP, night_c_evap4, day_c_evap4)
    rf_CO2_Air_ppm = def_rf_CO2_Air_ppm(c_evap3, eta_mg_ppm, CO2_Air)
    rf_VP_Can_VP_Air = def_rf_VP_Can_VP_Air(c_evap4, VP_Can, VP_Air)
    r_s = def_r_s(r_s_min, rf_R_can, rf_CO2_Air_ppm, rf_VP_Can_VP_Air)
    VEC_CanAir = def_VEC_CanAir(rho_Air, c_p_Air, LAI, deltaH, gamma, r_b, r_s)
    MV_CanAir = def_MV_CanAir(VEC_CanAir, VP_Can, VP_Air)
    print("Mv_CanAir = ", MV_CanAir)
    print()
    
    MV_PadAir = def_MV_PadAir(rho_Air, f_Pad, eta_Pad, x_Pad, x_Out)
    print("MV_PadAir = " MV_PadAir)
    print()
    
    MV_AirOut_Pad = def_MV_AirOut_Pad(f_Pad, M_water, VP_Air, R, T_Air)
    print("MV_AirOut_Pad = ", MV_AirOut_Pad)
    print()
    
    MV_FogAir = def_MV_FogAir(U_Fog, phi_Fog, A_Flr)
    print("MV_FogAir = ", MV_FogAir)
    print()
    
    MV_BlowAir = def_MV_BlowAir(eta_HeatVap, U_Blow, P_Blow, A_Flr)
    print("MV_BlowAir = ", MV_BlowAir)
    print()
    
    HEC_AirThScr = def_HEC_AirThScr(U_ThScr, T_Air, T_ThScr)
    MV_AirThScr = def_MV_AirThScr(HEC_AirThScr, VP_Air, VP_ThScr)
    print("MV_AirThScr = ", MV_AirThScr)
    print()
    
    HEC_AirMech = def_HEC_AirMech(U_MechCool, COP_MechCool, P_MechCool, A_Flr, T_Air, T_MechCool, deltaH, VP_Air, VP_MechCool)
    MV_AirMech = def_MV_AirMech(HEC_AirMech, VP_Air, VP_Mech)
    print("MV_AirMech = ", MV_AirMech)
    print()
    
    HEC_TopCov_in = def_HEC_TopCov_in(c_HECin, T_Top, T_Cov_in, A_Cov, A_Flr)
    MV_TopCov_in = def_MV_TopCov_in(HEC_TopCov_in, VP_Top, VP_Cov_in)
    print("MV_TopCov_in = ", MV_TopCov_in)
    print()
    
    f_ThScr = def_f_ThScr()
    MV_AirTop = def_MV_AirTop(M_water, f_ThScr, VP_Air, VP_Top, R, T_Air, T_Top)
    print("MV_AirTop = ", MV_AirTop)
    print()
    
    f_ventSide = def_f_ventSide()
    f_ventForced = def_f_ventForced()
    MV_AirOut = def_MV_AirOut(M_water, f_ventSide, f_ventForced, VP_Air, VP_Out, R, T_Air, T_Out)
    print("MV_AirOut = ", MV_AirOut)
    print()
    
    f_ventRoof = def_f_ventRoof()
    MV_TopOut = def_MV_TopOut(M_water, f_ventRoof, VP_Top, VP_Out, R, T_Top, T_Out)
    print("MV_TopOut = ", MV_TopOut)
    print()
    
    dx1 = (MV_CanAir + MV_PadAir + MV_BlowAir + MV_FogAir - MV_AirThScr - MV_AirTop - MV_AirOut - MV_AirOut_Pad - MV_AirMech)/cap_VP_Air
    dx2 = (MV_Airtop - MV_TopCov_in - MV_TopOut)/cap_VP_Top
    
    return [dx1, dx2]

In [ ]:
def def_rf_R_Can(R_Can, c_evap1, c_evap2):
    return (R_Can + c_evap1)/(R_Can + c_evap2)
def def_c_evap3(R_Can, R_Can_SP, night_c_evap3, day_c_evap3):
    if (R_Can >= R_Can_SP):
        return day_c_evap3
    else:
        return night_c_evap3
def def_c_evap4(R_Can, R_Can_SP, night_c_evap4, day_c_evap4):
    if (R_Can >= R_Can_SP):
        return day_c_evap4
    else:
        return night_c_evap4
def def_rf_CO2_Air_ppm(c_evap3, eta_mg_ppm, CO2_Air):
    return (1+c_evap3*(eta_mg_ppm*CO2_Air-200)**(2))
def def_rf_VP_Can_VP_Air(c_evap4, VP_Can, VP_Air):
    return (1+c_evap4*(VP_Can-VP_Air)**(2))
def def_r_s(r_s_min, rf_R_Can, rf_CO2_Air_ppm, rf_VP_Can_VP_Air):
    return (r_s_min*rf_R_Can*rf_CO2_Air_ppm*rf_VP_Can_VP_Air)
def def_VEC_CanAir(rho_Air, c_p_Air, LAI, deltaH, gamma, r_b, r_s):
    return (2*rho_Air*c_p_Air*LAI)/(deltaH*(r_b+r_s))
def def_MV_CanAir(VEC_CanAir, VP_Can, VP_Air):
    return (VEC_CanAir*(VP_Can-VP_Air))

def def_MV_PadAir(rho_Air, f_Pad, eta_Pad, x_Pad, x_Out):
    return (rho_Air*f_Pad*(eta_Pad*(x_Pad-x_Out)))

def def_MV_AirOut_Pad(f_Pad, M_water, VP_Air, R, T_Air):
    return (f_Pad*(M_water/R)*(VP_Air/(T_Air+273.15)))

def def_MV_FogAir(U_Fog, phi_Fog, A_Flr):
    return (U_Fog*phi_Fog/A_Flr)

def def_MV_BlowAir(eta_HeatVap, U_Blow, P_Blow, A_Flr):
    return ((eta_HeatVap*U_Blow*P_Blow)/A_Flr)

def def_HEC_AirThScr(U_ThScr, T_Air, T_ThScr):
    return (1.7*U_ThScr*abs(T_Air-T_ThScr)**(0.33))
def def_MV_AirThScr(HEC_AirThScr, VP_Air, VP_ThScr):
    if (VP_Air <= VP_ThScr):
        return 0
    else:
        return (6.4*(10**(-9)*HEC_AirThScr*(VP_Air-VP_ThScr)))

def def_HEC_AirMech(U_MechCool, COP_MechCool, P_MechCool, A_Flr, T_Air, T_MechCool, deltaH, VP_Air, VP_MechCool):
    return (U_MechCool*COP_MechCool*P_MechCool/A_Flr)/(T_Air-T_MechCool + 6.4*(10**(-9))*deltaH*(VP_Air-VP_MechCool))
def def_MV_AirMech(HEC_AirMech, VP_Air, VP_Mech):
    if (VP_Air <= VP_Mech):
        return 0
    else:
        return (6.4*(10**(-9))*HEC_AirMech*(VP_Air-VP_Mech))

def def_HEC_TopCov_in(C_HECin, T_Top, T_Cov_in, A_Cov, A_Flr):
    return (c_HECin*((T_Top-T_Cov,in)**(0.33))*A_Cov/A_Flr)
def def_MV_TopCov_in(HEC_TopCov_in, VP_Top, VP_Cov_in):
    if (VP_Top <= VP_Cov_in):
        return 0
    else:
        return (6.4*(10**(-9))*HEC_TopCov_in*(VP_Top-VP_Cov_in))
    
def def_f_ThScr()
def def_MV_AirTop(M_water, f_ThScr, VP_Air, VP_Top, R, T_Air, T_Top):
    return (M_water/R)*f_ThScr*((VP_Air/(T_Air+273.15))-(VP_Top/(T_Top+273.15)))

def def_f_ventSide()
def def_f_ventForced()
def def_MV_AirOut(M_water, f_ventSide, f_ventForced, VP_Air, VP_Out, R, T_Air, T_Out):
    return (M_water/R)*(f_ventSide+f_ventForced)*((VP_Air/(T_Air+273.15))-(VP_Out/(T_Out+273.15)))

def def_f_ventRoof()
def def_MV_TopOut(M_water, f_ventRoof, VP_Top, VP_Out, R, T_Top, T_Out):
    return (M_water/R)*f_ventRoof*((VP_Top/(T_Top+273.15))-(VP_Out/(T_Out+273.15)))

def def_cap_VP_Air(M_water, h_Air, R, T_Air):
    return ((M_water*h_Air)/(R*(T_Air+273.15)))
def def_cap_VP_Top(M_water, h_Gh, R, T_Top):
    return ((M_water*h_Air)/(R*(T_Top+273.15)))